In [ ]:
import pyDOE as pd
import numpy as np
#from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
import copy as cp
import GPy
import pickle
from scipy.optimize import Bounds
from scipy.optimize import minimize
#import torch
#import torch.nn as nn
#import torch.optim as optim
#import gptorch
from sklearn.model_selection import train_test_split
#import black_box as bb
from matplotlib.ticker import FormatStrFormatter
import matplotlib.ticker as ticker

In [ ]:
# Plotting size
plt.rc('xtick', labelsize=14) 
plt.rc('ytick', labelsize=14) 

In [ ]:
def scale(col, min, max):
    range = col.max() - col.min()
    a = (col - col.min()) / range
    return a * (max - min) + min

def scalex(col, min, max, xmax, xmin):
    range = xmax - xmin
    a = (col - xmin) / range
    return a * (max - min) + min

def rescale(xntd,min,max,x):
    xall = np.vstack((xntd,x))
    xalls = np.zeros((xall.shape[0],xall.shape[1]))
    
    # Scale values between min and max
    for i in range(xalls.shape[1]):
        xalls[:,i] = scalex(xall[:,i],min,max,x[:,i].max(),x[:,i].min())
        #xalls[:,i] = scale(xall[:,i],min,max)

    xntds = np.zeros((xntd.shape[0],xntd.shape[1]))

    for i in range(xntds.shape[1]):
        xntds[:,i] = xalls[0:(xntd.shape[0]),i]
        
    return xntds
    
def checkBound(xntd,x):
    for i in range(0,x.shape[1]):
        if xntd[:,i].max() > x[:,i].max():
            print("Max value for parameter in index %d out of training bounds."%(i))
        if xntd[:,i].min() < x[:,i].min():
            print("Min value for parameter in index %d out of training bounds."%(i))
            

## Load data

In [ ]:
# # ===================================================== #
# #                       Load data                       #
# # ===================================================== #
l0 = 0.2
v0 = l0**3.
loc = "/scratch/brown/jbarsima/RVE/RVE_adip_size/"

# Load deformed configuration
X = np.loadtxt(loc+'RVE_Psi.csv', delimiter=',', unpack=True) # Material parameters probed
X = X.transpose()

# Load deformed + eps configuration
Xplus = np.loadtxt(loc+'RVE_Psi_plus.csv', delimiter=',', unpack=True) # Material parameters probed
Xplus = Xplus.transpose()


# Load deformed - eps configuration
Xminus = np.loadtxt(loc+'RVE_Psi_minus.csv', delimiter=',', unpack=True) # Material parameters probed
Xminus = Xminus.transpose()


param_n = 4

#X: D, Dstd, 1-S, Sstd, Psi, Psi Vol, Psi Area, Psi edge, Volume, Cells

x = X[:,0:param_n] #D, Dstd, S, Sstd

x[:,2] = 1 - x[:,2] #Converting the value to sphericity (original value is 1 - sphericity)
psi = X[:,4] #Psi total
psi = psi.reshape(-1,1)
Vf = X[:,-2].reshape(-1,1) #Actual final volume
ncell = X[:,-1].reshape(-1,1)

xplus = Xplus[:,0:4] #D, Dstd, S, Sstd
xplus[:,2] = 1 - xplus[:,2] 
psi_plus = Xplus[:,4] #Psi plus total
psi_plus = psi_plus.reshape(-1,1)
ncell = Xplus[:,-1].reshape(-1,1)

xminus = Xminus[:,0:4] #D, Dstd, S, Sstd
xminus[:,2] = 1 - xminus[:,2] 
psi_minus = Xminus[:,4] #Psi minus total
psi_minus = psi_minus.reshape(-1,1)
Vf = Xminus[:,-2].reshape(-1,1) #Actual final volume
ncell = Xminus[:,-1].reshape(-1,1)

In [ ]:
# Load deformed - eps configuration
samples = np.loadtxt('/scratch/brown/jbarsima/RVE/RVE_adip_size/id1/samples.csv', delimiter=',', unpack=True) # Material parameters probed
samples = samples.transpose()

In [ ]:
plt.plot(samples[:,1],samples[:,2])

## Stress

In [ ]:
eps = 10.**-6
Vi = 0.5**3
#J = Vf/Vi
#lam = J**(1./3.)
lam = 1.15
J = lam**3
len_minus = len(psi_minus)
dpsidlam = (psi_plus[:len_minus] - psi_minus)/(2*eps) #Central difference
y = J**-1*lam*dpsidlam #Stress
y

In [ ]:
plt.plot((ncell/0.5**3)**1,y,'o')
plt.xlabel('Cell density [$(cells/{mm^{3}})^{0.5}$]')
plt.ylabel('Stress [MPa]')
plt.show()

plt.plot(xminus[:,0]**-0.5,y,'o')
plt.xlabel('Cell size, $d^{-0.5}$ [$mm^{-0.5}$]')
plt.ylabel('Stress [MPa]')
plt.show()

print((ncell).max())
print(np.mean(ncell/0.5**3))

## Train GP

In [ ]:
# Scale X values between a and b
a = 0.
b = 1.
xs = np.zeros((x.shape[0],x.shape[1]))

for i in range(xs.shape[1]):
    xs[:,i] = scale(x[:,i],a,b)

Xtrain, Xtest, ytrain, ytest = train_test_split(xs, y, train_size=0.60, random_state=27)

## GP

In [ ]:
# RBF
ls = 0.5
kernel = GPy.kern.RBF(input_dim=xs.shape[1], ARD=True, lengthscale=[ls,ls,ls,ls]) #When using scaled values
#print(k)

# Set up model
m = GPy.models.GPRegression(Xtrain,ytrain,kernel,mean_function=None)
#print(m_count)

# Plot the unfit version
#fig = m.plot()

m.optimize(messages=True,max_f_eval=1000)
#m.optimize_restarts(num_restarts = 10)
print(m)

In [ ]:
m.kern.lengthscale
#C = kernel.K(Xtest,Xtest)
#plt.matshow(C)

In [ ]:
ypred,var = m.predict(Xtest)

j = 0

yp = ypred + 1.96*np.sqrt(var)
ym = ypred - 1.96*np.sqrt(var)

print(np.sqrt(np.sum(ypred-ytest)**2/len(ypred)))


In [ ]:
# Define the input array Xtest
Xcont = cp.deepcopy(Xtest[6,:])
n = 0
k = 2

Ns = 30

# Create a contour plot based on the model predictions for Xtest
dx = np.linspace(a, b, Ns)
dy = np.linspace(a, b, Ns)
Xplot = np.linspace(x[:,n].min(),x[:,n].max(),Ns)
Yplot = np.linspace(x[:,k].min(),x[:,k].max(),Ns)
X, Y = np.meshgrid(dx, dy)
DX, DY = np.meshgrid(Xplot, Yplot)
Z = np.zeros((len(dx), len(dy)))

for i in range(len(dx)):
    for j in range(len(dy)):
        #Xcont[n] = Xplot[i]
        #Xcont[k] = Yplot[j]
        Xcont[n] = dx[i]
        Xcont[k] = dy[j]
        ycontour, oo = m.predict(Xcont.reshape(1,4))
        Z[i, j] = ycontour

plt.contourf(DX, DY, Z, cmap='plasma')
cb = plt.colorbar(format=ticker.LogFormatterSciNotation(), label='Stress [MPa]')
cb.ax.yaxis.set_minor_formatter(ticker.NullFormatter())
plt.xlabel('Diameter')
plt.ylabel('Sphericity')
plt.show()

In [ ]:
Xcont

In [ ]:
Xtest[1,:]